In [485]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import warnings
import xgboost as xgb
import lightgbm as lgb
from scipy.stats import skew
from scipy import stats
from scipy.stats.stats import pearsonr
from scipy.stats import norm
from collections import Counter

import sklearn
import sklearn.model_selection
import sklearn.ensemble
import sklearn.metrics
import sklearn.model_selection

In [486]:
train = pd.read_csv('Train.csv.zip', compression='zip')
test = pd.read_csv('Test.csv.zip', compression='zip')
# train.columns

In [487]:
# Save the 'Id' column
train_ID = train['id']
test_ID = test['id']

# Now drop the 'Id' column since it's unnecessary for the prediction process.
train.drop("id", axis = 1, inplace = True)
test.drop("id", axis = 1, inplace = True)

In [488]:
# Combining Datasets
ntrain = train.shape[0]
ntest = test.shape[0]
y_train = train.price.values
all_data = pd.concat((train, test)).reset_index(drop=True)
all_data.drop(['price'], axis=1, inplace=True)
print("Train data size is : {}".format(train.shape))
print("Test data size is : {}".format(test.shape))
print("Combined dataset size is : {}".format(all_data.shape))

Train data size is : (100000, 24)
Test data size is : (100000, 23)
Combined dataset size is : (200000, 23)


/home/axreldable/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


In [489]:
# настройка параметров
all_data[['build_tech']] = all_data[['build_tech']].replace(np.nan, 0.5)
all_data[['metro_dist']] = all_data[['metro_dist']].replace(np.nan, 15.0)
train[['g_lift']] = train[['g_lift']].replace(np.nan, 0.5)

In [490]:
all_data.drop("date", axis = 1, inplace = True)
all_data.drop("kw1", axis = 1, inplace = True)
all_data.drop("kw2", axis = 1, inplace = True)
all_data.drop("kw3", axis = 1, inplace = True)
all_data.drop("kw4", axis = 1, inplace = True)
all_data.drop("kw5", axis = 1, inplace = True)
all_data.drop("kw6", axis = 1, inplace = True)
all_data.drop("kw7", axis = 1, inplace = True)
all_data.drop("kw8", axis = 1, inplace = True)
all_data.drop("kw9", axis = 1, inplace = True)
all_data.drop("kw10", axis = 1, inplace = True)
all_data.drop("kw11", axis = 1, inplace = True)
all_data.drop("kw12", axis = 1, inplace = True)
all_data.drop("kw13", axis = 1, inplace = True)

In [491]:
# Check if there are any missing values left
all_data_na = (all_data.isnull().sum() / len(all_data)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data.head()

,Missing Ratio
g_lift,29.8705


In [492]:
all_data = pd.get_dummies(all_data)
print(all_data.shape)

(200000, 9)


In [493]:
all_data
all_data.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 9 columns):
area          200000 non-null int64
balcon        200000 non-null int64
build_tech    200000 non-null float64
floor         200000 non-null int64
g_lift        140259 non-null float64
metro_dist    200000 non-null float64
n_photos      200000 non-null int64
rooms         200000 non-null int64
street_id     200000 non-null int64
dtypes: float64(3), int64(6)
memory usage: 13.7 MB


In [494]:
# all_data

In [495]:
train = all_data[:ntrain]
test = all_data[ntrain:]

In [496]:
y_train

array([ 3947000,  2525000, 19984000, ...,  3076000,  8251000,  2346000])

In [497]:
X, Xt, y, yt = sklearn.model_selection.train_test_split(train, y_train, test_size=0.3, random_state=0)

In [498]:
# X

In [499]:
model_xgb = xgb.XGBRegressor(max_depth=13, n_estimators=10000, random_state =7)

# XGBRegressor max_depth=6 n_estimators=100: 1675995.2668916667
# XGBRegressor max_depth=6 n_estimators=5000: 959617.0378790039
# XGBRegressor max_depth=6 n_estimators=10000: 911419.2695312988
# XGBRegressor max_depth=6 n_estimators=15000: 902241.6986599203

# XGBRegressor max_depth=6 n_estimators=100: 1282773.6694083333 заполненный g_lift
# XGBRegressor max_depth=6 n_estimators=100: 1284701.1225666667 пустой g_lift
# XGBRegressor max_depth=6 n_estimators=100: 1294787.9690958334 с датой как dummies
# XGBRegressor max_depth=6 n_estimators=100: 1285825.3127229167 с включенным kw10

# XGBRegressor max_depth=11 n_estimators=100: 1031950.7442583333
# XGBRegressor max_depth=12 n_estimators=100: 1011165.3294583333
# XGBRegressor max_depth=13 n_estimators=100: 1009005.9016822916
# XGBRegressor max_depth=14 n_estimators=100: 1027168.0148854167
# XGBRegressor max_depth=24 n_estimators=100: 1270361.10094375

# XGBRegressor max_depth=13 n_estimators=5000: 864670.0302248698
# XGBRegressor max_depth=13 n_estimators=10000: 864670.2538933593

In [500]:
model_xgb.fit(X, y)

pred = model_xgb.predict(Xt)

score = sklearn.metrics.mean_absolute_error(yt, pred)

print('XGBRegressor max_depth=13 n_estimators=10000: %s' % (score))

XGBRegressor max_depth=13 n_estimators=10000: 864670.2538933593


In [501]:
# model_xgb.fit(train, y_train)

# pred_rez = clf_gbm.predict(test)

In [502]:
# test['price'] = pred_rez
# test['id'] = test_ID

# test[['id', 'price']].to_csv('sub.csv', index=False)